# RNA Structure Benchmark Preprocessing & Utilities

This notebook:

1. Converts **ArchiveII** `.bpseq` files → `sequence, dot_bracket` CSV  
2. Converts **bpRNA-NF-15.0** raw CSV → the same format  
3. Implements encoding utilities (sequence & dot-bracket)  
4. Implements PyTorch Dataset classes + collate function  
5. Implements Nussinov dynamic programming decoder  
6. Runs sanity checks on both datasets


In [1]:
#Import necessary libraries
import os
import glob
from typing import List, Tuple

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

print("PyTorch version:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
device


PyTorch version: 2.9.1+cpu


'cpu'

In [2]:
from pathlib import Path

# This notebook lives in project_root/notebooks
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

DATA_DIR = PROJECT_ROOT / "data"

print("Notebook dir:", NOTEBOOK_DIR)
print("Project root:", PROJECT_ROOT)
print("Data dir:", DATA_DIR)


Notebook dir: c:\Fall_2025\CSE6301\Project\rna-structural-prediction\notebooks
Project root: c:\Fall_2025\CSE6301\Project\rna-structural-prediction
Data dir: c:\Fall_2025\CSE6301\Project\rna-structural-prediction\data


## 1. ArchiveII – BPSEQ → Standardized CSV

We know the BPSEQ files are in:

`data/archiveII/raw/archiveII_bpseq/archiveII_bpseq/`

We will parse all `.bpseq` files, extract:

- `sequence_id` – file name without extension  
- `sequence` – RNA sequence (T converted to U)  
- `dot_bracket` – structure inferred from BPSEQ pairs  

and save to:

`data/archiveII/processed/archiveII_from_bpseq.csv`


In [3]:
# Set up ArchiveII BPSEQ paths (relative to this notebook's working directory)
archive_bpseq_dir = DATA_DIR / "archiveII" / "raw" / "archiveII_bpseq" / "archiveII_bpseq"
archive_processed_dir = DATA_DIR / "archiveII" / "processed"
archive_processed_dir.mkdir(parents=True, exist_ok=True)

archiveii_csv_path = archive_processed_dir / "archiveII_from_bpseq.csv"

archive_bpseq_dir, archive_processed_dir, archiveii_csv_path


(WindowsPath('c:/Fall_2025/CSE6301/Project/rna-structural-prediction/data/archiveII/raw/archiveII_bpseq/archiveII_bpseq'),
 WindowsPath('c:/Fall_2025/CSE6301/Project/rna-structural-prediction/data/archiveII/processed'),
 WindowsPath('c:/Fall_2025/CSE6301/Project/rna-structural-prediction/data/archiveII/processed/archiveII_from_bpseq.csv'))

In [4]:
def parse_bpseq_file(bpseq_path: str) -> Tuple[str, str]:
    """
    Parse a BPSEQ file and return (sequence, dot_bracket).

    BPSEQ format (typical):
        index  base  pair_index
    - index: 1-based position
    - base:  single character (A,C,G,U/T,...)
    - pair_index: 1-based index of paired position, 0 or -1 if unpaired.
    """
    indices = []
    bases = []
    partners = []

    with open(bpseq_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("#") or line.startswith(";"):
                continue

            parts = line.split()
            if len(parts) < 3:
                continue

            idx = int(parts[0])
            base = parts[1]
            pair_idx = int(parts[2])

            indices.append(idx)
            bases.append(base)
            partners.append(pair_idx)

    # sort by index just in case
    order = sorted(range(len(indices)), key=lambda k: indices[k])
    bases = [bases[i] for i in order]
    partners = [partners[i] for i in order]

    L = len(bases)

    # Build list of base pairs (0-based)
    pairs: List[Tuple[int, int]] = []
    for i in range(L):
        p = partners[i]
        if p <= 0:
            continue
        j = p - 1  # convert to 0-based
        if j > i:
            pairs.append((i, j))

    # Convert to dot-bracket
    chars = ["."] * L
    for i, j in pairs:
        chars[i] = "("
        chars[j] = ")"
    dot_bracket = "".join(chars)

    # Normalize sequence: T -> U
    seq = "".join(bases).upper().replace("T", "U")

    return seq, dot_bracket


In [5]:
bpseq_files = list(archive_bpseq_dir.glob("*.bpseq"))
print(f"Found {len(bpseq_files)} BPSEQ files in {archive_bpseq_dir}")

archive_records = []

for path in bpseq_files:
    fname = path.name
    seq_id = os.path.splitext(fname)[0]

    seq, db = parse_bpseq_file(path)

    if len(seq) != len(db):
        print(
            f"WARNING: length mismatch in {fname}: "
            f"len(seq)={len(seq)}, len(dot_bracket)={len(db)}"
        )

    archive_records.append(
        {
            "sequence_id": seq_id,
            "sequence": seq,
            "dot_bracket": db,
        }
    )

archiveii_df = pd.DataFrame(archive_records)
archiveii_df.to_csv(archiveii_csv_path, index=False)

print("ArchiveII standardized CSV saved to:", archiveii_csv_path)
archiveii_df.head()


Found 3975 BPSEQ files in c:\Fall_2025\CSE6301\Project\rna-structural-prediction\data\archiveII\raw\archiveII_bpseq\archiveII_bpseq
ArchiveII standardized CSV saved to: c:\Fall_2025\CSE6301\Project\rna-structural-prediction\data\archiveII\processed\archiveII_from_bpseq.csv


,sequence_id,sequence,dot_bracket
0,16s_A.fulgidus,AUUCUGGUUGAUCCUGCCAGAGGCCGCUGCUAUCCGGCUGGGACUA...,...(((((...(((.))))).((((((((((.((((((((((.......
1,16s_A.fulgidus_domain1,AUUCUGGUUGAUCCUGCCAGAGGCCGCUGCUAUCCGGCUGGGACUA...,...(((((.......))))).((((((((((.((((((((((.......
2,16s_A.fulgidus_domain2,UUUAUUGGGCCUAAAGCGUCCGUAGCCGGGCUGGUAAGUCCUCCGG...,.......(((((...(.((((.(.(((.(((((((.((((((((((...
3,16s_A.fulgidus_domain3,AAGGAAUUGGCGGGGGAGCACUACAACGGGUGGAGCCUGCGGUUUA...,.......(((((.(((((((..((..((((((.((((((((((......
4,16s_A.fulgidus_domain4,ACCGCCCGUCAAGCCACCCGAGUGGGCCAGGGGCGAGGGGGUGGCC...,.(.(..((...((((.(((..(((((((..((((..((((((((((...


## 2. bpRNA-NF-15.0 – Raw CSV → Standardized CSV

Raw file path (you may have both absolute and relative):

- Absolute (Windows, from your system):  
  `C:\Fall_2025\CSE6301\Project\rna-structural-prediction\data\bprna_nf15\raw\bpRNA-NF-15.0.csv`

We’ll also define a **relative path** so the notebook is portable:

`data/bprna_nf15/raw/bpRNA-NF-15.0.csv`

We will inspect its columns, then map:

- `sequence_id`
- `sequence`
- `dot_bracket`

into:

`data/bprna_nf15/processed/bprna_nf15_standardized.csv`


In [6]:
# Relative to project root (data is sibling of notebooks)
bprna_raw_csv = DATA_DIR / "bprna_nf15" / "raw" / "bpRNA-NF-15.0.csv"

bprna_processed_dir = DATA_DIR / "bprna_nf15" / "processed"
bprna_processed_dir.mkdir(parents=True, exist_ok=True)

bprna_processed_csv = bprna_processed_dir / "bprna_nf15_standardized.csv"

bprna_raw_csv, bprna_processed_csv


(WindowsPath('c:/Fall_2025/CSE6301/Project/rna-structural-prediction/data/bprna_nf15/raw/bpRNA-NF-15.0.csv'),
 WindowsPath('c:/Fall_2025/CSE6301/Project/rna-structural-prediction/data/bprna_nf15/processed/bprna_nf15_standardized.csv'))

In [7]:
bprna_df_raw = pd.read_csv(bprna_raw_csv)

print("bpRNA-NF-15.0 raw shape:", bprna_df_raw.shape)
print("Columns:", list(bprna_df_raw.columns))
bprna_df_raw.head()


bpRNA-NF-15.0 raw shape: (8788, 3)
Columns: ['rna_name', 'seq', 'struct']


,rna_name,seq,struct
0,CP002582.1/3780440-3780356,UUAAUGAGAAUAAAUCAGGUUAUUAUGUUGGAUGUAAUAACUGGGC...,.........[[[...[.((((((((((.....))))))))))..((...
1,CP001997.1/495019-495096,UCCUUACAAGAUCAGGCGGCUUUUAGUGCGGCCGGGCCUCUUUUGA...,.......[[[..[.(((.((.....))...)))..(((((....))...
2,CP002028.1/1768652-1768573,AAAGCAUAAUGGAUCAGGUAGAACGGAAAAGUUUUACCGGGCUCUU...,........[[[[..[.((((((((......))))))))..((((.....
3,AAWL01000017.1/25112-25035,AUAAUACAACAGGAACAGGCGGCUGCCGCGAGGCAGUGCAGCCGGG...,.........[[[[..[.(((.((((((....))))))...)))..(...
4,ADLR01000090.1/3100-3003,AUAUGAUACAGUGAGCAGGCGACGGCCGAAAUGAGAAUCGUCGCCG...,.........[.[[..[.((((((((............))))))))....


In [8]:
# TODO: adjust these three names to match actual column names in bpRNA-NF-15.0.csv
# Common patterns: 'sequence', 'str', 'structure', 'dot_bracket', etc.
seq_col = "seq"   # <-- change if different
db_col = "struct"   # <-- change if different
id_col = "rna_name"          # <-- change if different; or we create synthetic IDs

# If the ID column doesn't exist, create a synthetic one
if id_col not in bprna_df_raw.columns:
    print(f"Column '{id_col}' not found; creating synthetic sequence_id.")
    bprna_df_raw["sequence_id"] = [f"bprna_{i}" for i in range(len(bprna_df_raw))]
    id_col = "sequence_id"

# Normalize sequences: T -> U
bprna_df_raw[seq_col] = bprna_df_raw[seq_col].astype(str).str.upper().str.replace("T", "U")

bprna_df = pd.DataFrame(
    {
        "sequence_id": bprna_df_raw[id_col].astype(str),
        "sequence": bprna_df_raw[seq_col].astype(str),
        "dot_bracket": bprna_df_raw[db_col].astype(str),
    }
)

len_mismatch = (bprna_df["sequence"].str.len() != bprna_df["dot_bracket"].str.len()).sum()
print("Standardized bpRNA shape:", bprna_df.shape)
print("Length mismatches:", len_mismatch)

bprna_df.head()


Standardized bpRNA shape: (8788, 3)
Length mismatches: 0


,sequence_id,sequence,dot_bracket
0,CP002582.1/3780440-3780356,UUAAUGAGAAUAAAUCAGGUUAUUAUGUUGGAUGUAAUAACUGGGC...,.........[[[...[.((((((((((.....))))))))))..((...
1,CP001997.1/495019-495096,UCCUUACAAGAUCAGGCGGCUUUUAGUGCGGCCGGGCCUCUUUUGA...,.......[[[..[.(((.((.....))...)))..(((((....))...
2,CP002028.1/1768652-1768573,AAAGCAUAAUGGAUCAGGUAGAACGGAAAAGUUUUACCGGGCUCUU...,........[[[[..[.((((((((......))))))))..((((.....
3,AAWL01000017.1/25112-25035,AUAAUACAACAGGAACAGGCGGCUGCCGCGAGGCAGUGCAGCCGGG...,.........[[[[..[.(((.((((((....))))))...)))..(...
4,ADLR01000090.1/3100-3003,AUAUGAUACAGUGAGCAGGCGACGGCCGAAAUGAGAAUCGUCGCCG...,.........[.[[..[.((((((((............))))))))....


In [9]:
bprna_df.to_csv(bprna_processed_csv, index=False)
print("Standardized bpRNA CSV saved to:", bprna_processed_csv)

Standardized bpRNA CSV saved to: c:\Fall_2025\CSE6301\Project\rna-structural-prediction\data\bprna_nf15\processed\bprna_nf15_standardized.csv


## 3. Encoding Utilities

We now define:

- `encode_sequence(seq)` → integer IDs (A/C/G/U)
- `dotbracket_to_pairs(dot_bracket)` → list of base pairs
- `pairs_to_matrix(pairs, length)` → L×L numpy matrix
- `dotbracket_to_matrix(dot_bracket)` → convenience wrapper


In [10]:
# Basic RNA vocabulary
BASE2IDX = {"A": 0, "C": 1, "G": 2, "U": 3}


def encode_sequence(seq: str) -> torch.Tensor:
    """
    Encode an RNA sequence to integer IDs.
    - Upper-case
    - Convert T->U
    - Unknown bases -> 0 (A)
    """
    seq = seq.upper().replace("T", "U")
    idxs = [BASE2IDX.get(b, 0) for b in seq]
    return torch.tensor(idxs, dtype=torch.long)


def dotbracket_to_pairs(dot_bracket: str) -> List[Tuple[int, int]]:
    """
    Convert dot-bracket string to base-pair list (i, j), 0-based.
    Assumes non-pseudoknotted structures using '(' and ')'.
    """
    stack = []
    pairs = []
    for i, ch in enumerate(dot_bracket):
        if ch == "(":
            stack.append(i)
        elif ch == ")":
            if not stack:
                raise ValueError(f"Unmatched ')' at position {i} in {dot_bracket}")
            j = stack.pop()
            pairs.append((j, i))
        # '.' etc. = unpaired

    if stack:
        raise ValueError(f"Unmatched '(' in dot-bracket: {dot_bracket}")

    return pairs


def pairs_to_matrix(pairs: List[Tuple[int, int]], length: int) -> np.ndarray:
    """
    Convert base-pair list to symmetric L×L matrix.
    """
    mat = np.zeros((length, length), dtype=np.float32)
    for i, j in pairs:
        if 0 <= i < length and 0 <= j < length:
            mat[i, j] = 1.0
            mat[j, i] = 1.0
    return mat


def dotbracket_to_matrix(dot_bracket: str) -> np.ndarray:
    """
    dot-bracket → base-pair list → L×L matrix
    """
    L = len(dot_bracket)
    pairs = dotbracket_to_pairs(dot_bracket)
    return pairs_to_matrix(pairs, L)


In [11]:
seq_test = "GCAUCU"
db_test = "..(..)"

print("Sequence:", seq_test)
print("Dot-bracket:", db_test)

encoded = encode_sequence(seq_test)
pairs = dotbracket_to_pairs(db_test)
mat = dotbracket_to_matrix(db_test)

print("Encoded:", encoded)
print("Pairs:", pairs)
print("Matrix:\n", mat)


Sequence: GCAUCU
Dot-bracket: ..(..)
Encoded: tensor([2, 1, 0, 3, 1, 3])
Pairs: [(2, 5)]
Matrix:
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


## 4. PyTorch Datasets and Collate Function

We define:

- `ArchiveIIDataset` – loads `archiveII_from_bpseq.csv`
- `BpRNANFDataset` – loads `bprna_nf15_standardized.csv`
- `collate_rna_batch` – pads sequences and matrices into batched tensors


In [12]:
class ArchiveIIDataset(Dataset):
    """
    ArchiveII dataset from standardized CSV with columns:
      - sequence_id
      - sequence
      - dot_bracket
    """
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
        self.df = pd.read_csv(csv_path)
        required = {"sequence", "dot_bracket"}
        if not required.issubset(self.df.columns):
            raise ValueError(f"Missing required columns {required} in {csv_path}")

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        seq = str(row["sequence"])
        db = str(row["dot_bracket"])
        mat = torch.from_numpy(dotbracket_to_matrix(db))  # (L, L)
        return seq, mat


class BpRNANFDataset(Dataset):
    """
    bpRNA-NF-15.0 dataset from standardized CSV with columns:
      - sequence_id
      - sequence
      - dot_bracket
    """
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
        self.df = pd.read_csv(csv_path)
        required = {"sequence", "dot_bracket"}
        if not required.issubset(self.df.columns):
            raise ValueError(f"Missing required columns {required} in {csv_path}")

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        seq = str(row["sequence"])
        db = str(row["dot_bracket"])
        mat = torch.from_numpy(dotbracket_to_matrix(db))
        return seq, mat


def collate_rna_batch(batch):
    """
    Collate function for variable-length RNA sequences.

    batch: list of (sequence_str, basepair_matrix_tensor (L, L))

    Returns:
      input_ids: (B, Lmax) int64
      mask:      (B, Lmax) bool (True = valid token)
      targets:   (B, Lmax, Lmax) float32
    """
    seqs, mats = zip(*batch)
    lengths = [len(s) for s in seqs]
    Lmax = max(lengths)
    B = len(seqs)

    input_ids = torch.zeros((B, Lmax), dtype=torch.long)
    mask = torch.zeros((B, Lmax), dtype=torch.bool)
    targets = torch.zeros((B, Lmax, Lmax), dtype=torch.float32)

    for i, (seq, mat) in enumerate(zip(seqs, mats)):
        L = len(seq)
        input_ids[i, :L] = encode_sequence(seq)
        mask[i, :L] = True
        targets[i, :L, :L] = mat

    return input_ids, mask, targets


In [13]:
# Paths to standardized CSVs we created earlier
archiveii_csv_path = archiveii_csv_path  # from Cell 4
bprna_processed_csv = bprna_processed_csv  # from Cell 8/11

arch_dataset = ArchiveIIDataset(archiveii_csv_path)
bprna_dataset = BpRNANFDataset(bprna_processed_csv)

print("ArchiveII dataset size:", len(arch_dataset))
print("bpRNA dataset size:", len(bprna_dataset))

arch_loader = DataLoader(arch_dataset, batch_size=4, shuffle=True, collate_fn=collate_rna_batch)

input_ids, mask, targets = next(iter(arch_loader))
print("input_ids shape:", input_ids.shape)
print("mask shape:", mask.shape)
print("targets shape:", targets.shape)


ArchiveII dataset size: 3975
bpRNA dataset size: 8788
input_ids shape: torch.Size([4, 342])
mask shape: torch.Size([4, 342])
targets shape: torch.Size([4, 342, 342])


In [14]:
import random

# For reproducibility
random.seed(42)

def make_eval_subset(dataset, max_len=400, max_n=200):
    """
    Filter dataset by sequence length and randomly sample at most max_n examples.

    Args:
      dataset: ArchiveIIDataset or BpRNANFDataset
      max_len: maximum sequence length allowed
      max_n:   maximum number of sequences to keep

    Returns:
      subset: torch.utils.data.Subset
    """
    # 1. Filter by length
    filtered_indices = [
        i for i in range(len(dataset))
        if len(dataset[i][0]) <= max_len
    ]
    print(f"Total = {len(dataset)}, length <= {max_len} = {len(filtered_indices)}")

    # 2. Randomly sample at most max_n
    if len(filtered_indices) > max_n:
        filtered_indices = random.sample(filtered_indices, max_n)
        print(f"Randomly sampled {max_n} indices for evaluation.")
    else:
        print("Using all filtered sequences (no subsampling).")

    subset = torch.utils.data.Subset(dataset, filtered_indices)
    return subset


# You can tweak these depending on runtime
ARCH_MAX_LEN = 400
ARCH_MAX_N   = 200

BPRNA_MAX_LEN = 400
BPRNA_MAX_N   = 200

arch_eval_subset = make_eval_subset(arch_dataset, max_len=ARCH_MAX_LEN, max_n=ARCH_MAX_N)
bprna_eval_subset = make_eval_subset(bprna_dataset, max_len=BPRNA_MAX_LEN, max_n=BPRNA_MAX_N)

print("ArchiveII eval subset size:", len(arch_eval_subset))
print("bpRNA eval subset size:", len(bprna_eval_subset))


Total = 3975, length <= 400 = 3685
Randomly sampled 200 indices for evaluation.
Total = 8788, length <= 400 = 8624
Randomly sampled 200 indices for evaluation.
ArchiveII eval subset size: 200
bpRNA eval subset size: 200


## 5. Nussinov Dynamic Programming Decoder

We implement:

- `nussinov_from_sequence(seq)` – classic DP maximizing number of base pairs  
- `nussinov_from_scores(seq, score_matrix)` – future use with neural model scores  


In [15]:
def can_pair(b1: str, b2: str) -> bool:
    """
    Canonical base-pairing rules (Watson-Crick + wobble).
    """
    pair = {b1 + b2, b2 + b1}
    return bool(pair & {"AU", "UA", "GC", "CG", "GU", "UG"})


def nussinov_dp(seq: str, scores: np.ndarray | None = None) -> np.ndarray:
    """
    Nussinov DP to maximize total pairing score.

    If scores is None, each valid pair scores 1.0.
    Otherwise, scores[i, j] is used for base pair (i, j).
    """
    L = len(seq)
    dp = np.zeros((L, L), dtype=np.float32)

    for length in range(1, L):  # j - i
        for i in range(L - length):
            j = i + length

            # Case 1: i unpaired
            best = dp[i + 1, j]

            # Case 2: j unpaired
            best = max(best, dp[i, j - 1])

            # Case 3: i paired with j
            if can_pair(seq[i], seq[j]):
                pair_score = 1.0 if scores is None else scores[i, j]
                best = max(best, dp[i + 1, j - 1] + pair_score)

            # Case 4: bifurcation
            for k in range(i + 1, j):
                best = max(best, dp[i, k] + dp[k + 1, j])

            dp[i, j] = best

    return dp


def nussinov_traceback(
    seq: str, dp: np.ndarray, scores: np.ndarray | None, i: int, j: int, pairs: List[Tuple[int, int]]
):
    """
    Traceback to recover optimal base pairs from DP table.
    """
    if i >= j:
        return

    if dp[i, j] == dp[i + 1, j]:
        nussinov_traceback(seq, dp, scores, i + 1, j, pairs)
    elif dp[i, j] == dp[i, j - 1]:
        nussinov_traceback(seq, dp, scores, i, j - 1, pairs)
    elif can_pair(seq[i], seq[j]):
        pair_score = 1.0 if scores is None else scores[i, j]
        if dp[i, j] == dp[i + 1, j - 1] + pair_score:
            pairs.append((i, j))
            nussinov_traceback(seq, dp, scores, i + 1, j - 1, pairs)
        else:
            for k in range(i + 1, j):
                if dp[i, j] == dp[i, k] + dp[k + 1, j]:
                    nussinov_traceback(seq, dp, scores, i, k, pairs)
                    nussinov_traceback(seq, dp, scores, k + 1, j, pairs)
                    break
    else:
        for k in range(i + 1, j):
            if dp[i, j] == dp[i, k] + dp[k + 1, j]:
                nussinov_traceback(seq, dp, scores, i, k, pairs)
                nussinov_traceback(seq, dp, scores, k + 1, j, pairs)
                break


def pairs_to_dotbracket(pairs: List[Tuple[int, int]], length: int) -> str:
    chars = ["."] * length
    for i, j in pairs:
        chars[i] = "("
        chars[j] = ")"
    return "".join(chars)


def nussinov_from_sequence(seq: str) -> str:
    """
    Classic Nussinov: each valid base pair = 1.
    """
    L = len(seq)
    if L == 0:
        return ""
    dp = nussinov_dp(seq, scores=None)
    pairs: List[Tuple[int, int]] = []
    nussinov_traceback(seq, dp, scores=None, i=0, j=L - 1, pairs=pairs)
    return pairs_to_dotbracket(pairs, L)


def nussinov_from_scores(seq: str, score_matrix: np.ndarray) -> str:
    """
    Nussinov decoding using model-provided score matrix.
    """
    L = len(seq)
    assert score_matrix.shape == (L, L)
    dp = nussinov_dp(seq, scores=score_matrix)
    pairs: List[Tuple[int, int]] = []
    nussinov_traceback(seq, dp, scores=score_matrix, i=0, j=L - 1, pairs=pairs)
    return pairs_to_dotbracket(pairs, L)


In [16]:
# Take first ArchiveII sequence and run Nussinov baseline
seq0, true_mat0 = arch_dataset[0]
pred_db0 = nussinov_from_sequence(seq0)

print("Sequence ID 0 length:", len(seq0))
print("Sequence 0:", seq0[:80] + ("..." if len(seq0) > 80 else ""))
print("Nussinov predicted structure (first 80 chars):", pred_db0[:80] + ("..." if len(pred_db0) > 80 else ""))

# Compare predicted structure matrix with true matrix shape
pred_pairs0 = dotbracket_to_pairs(pred_db0)
pred_mat0 = pairs_to_matrix(pred_pairs0, len(seq0))

print("True matrix shape:", true_mat0.shape)
print("Pred matrix shape:", pred_mat0.shape)


Sequence ID 0 length: 1492
Sequence 0: AUUCUGGUUGAUCCUGCCAGAGGCCGCUGCUAUCCGGCUGGGACUAAGCCAUGCGAGUCAAGGGGCUUGUAUCCCUUCGG...
Nussinov predicted structure (first 80 chars): .(((())(())((((()(.)((()()))()(()(())(())(((().()(()()).))))))))()((((()(((((())...
True matrix shape: torch.Size([1492, 1492])
Pred matrix shape: (1492, 1492)


In [17]:
def compare_pair_matrices(true_mat, pred_mat):
    """
    Computes pair-level TP, FP, FN, Precision, Recall, F1.

    Inputs:
      true_mat: torch.Tensor or numpy array (L, L)
      pred_mat: same shape

    Returns:
      dictionary of metrics
    """
    # Convert to torch tensors
    if not isinstance(true_mat, torch.Tensor):
        true_mat = torch.tensor(true_mat)
    if not isinstance(pred_mat, torch.Tensor):
        pred_mat = torch.tensor(pred_mat)

    # Only upper triangle is needed (avoid counting (i,j) and (j,i) twice)
    triu = torch.triu(torch.ones_like(true_mat), diagonal=1).bool()

    true_pairs = true_mat.bool() & triu
    pred_pairs = pred_mat.bool() & triu

    TP = (true_pairs & pred_pairs).sum().item()
    FP = ((~true_pairs) & pred_pairs).sum().item()
    FN = (true_pairs & (~pred_pairs)).sum().item()

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall    = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1        = 2*TP / (2*TP + FP + FN) if (2*TP + FP + FN) > 0 else 0.0

    return {
        "TP": TP,
        "FP": FP,
        "FN": FN,
        "Precision": precision,
        "Recall": recall,
        "F1": f1,
    }


In [18]:
def run_nussinov_eval_on_subset(subset, name="dataset"):
    """
    Run Nussinov on every sequence in the given subset and collect metrics.

    subset: torch.utils.data.Subset (or any Dataset with __getitem__ -> (seq, true_mat))
    name:   label for logging

    Returns:
      metrics_df: pandas DataFrame with one row per sequence
    """
    records = []
    n = len(subset)
    print(f"Running Nussinov on {n} sequences from {name}...")

    for i in range(n):
        seq, true_mat = subset[i]   # seq: str, true_mat: torch.Tensor (L, L)
        L = len(seq)

        # Run Nussinov
        pred_db = nussinov_from_sequence(seq)
        pred_pairs = dotbracket_to_pairs(pred_db) if "(" in pred_db or ")" in pred_db else []
        pred_mat = pairs_to_matrix(pred_pairs, L)
        pred_mat_t = torch.tensor(pred_mat, dtype=torch.float32)

        # Compute metrics
        metrics = compare_pair_matrices(true_mat, pred_mat_t)
        metrics["seq_len"] = L
        metrics["index"] = i

        records.append(metrics)

        if (i + 1) % 20 == 0 or (i + 1) == n:
            print(f"  Done {i+1}/{n}")

    df = pd.DataFrame(records)
    print(f"\n{name} Nussinov summary:")
    print(df[["Precision", "Recall", "F1"]].describe())
    return df


In [19]:
# Nussinov evaluation on ArchiveII eval subset
arch_nussinov_df = run_nussinov_eval_on_subset(arch_eval_subset, name="ArchiveII")

# Nussinov evaluation on bpRNA eval subset
bprna_nussinov_df = run_nussinov_eval_on_subset(bprna_eval_subset, name="bpRNA-NF-15.0")


Running Nussinov on 200 sequences from ArchiveII...
  Done 20/200
  Done 40/200
  Done 60/200
  Done 80/200
  Done 100/200
  Done 120/200
  Done 140/200
  Done 160/200
  Done 180/200
  Done 200/200

ArchiveII Nussinov summary:
        Precision      Recall          F1
count  200.000000  200.000000  200.000000
mean     0.032028    0.052749    0.039553
std      0.057043    0.094773    0.070494
min      0.000000    0.000000    0.000000
25%      0.000000    0.000000    0.000000
50%      0.000000    0.000000    0.000000
75%      0.042052    0.076406    0.052731
max      0.317073    0.550000    0.354839
Running Nussinov on 200 sequences from bpRNA-NF-15.0...
  Done 20/200
  Done 40/200
  Done 60/200
  Done 80/200
  Done 100/200
  Done 120/200
  Done 140/200
  Done 160/200
  Done 180/200
  Done 200/200

bpRNA-NF-15.0 Nussinov summary:
        Precision      Recall          F1
count  200.000000  200.000000  200.000000
mean     0.037279    0.074482    0.048184
std      0.079876    0.134916    0

In [20]:
from pathlib import Path

results_dir = Path("../results")
results_dir.mkdir(parents=True, exist_ok=True)

arch_nussinov_df.to_csv(results_dir / "archiveII_nussinov_eval.csv", index=False)
bprna_nussinov_df.to_csv(results_dir / "bprna_nussinov_eval.csv", index=False)

print("Saved Nussinov evaluation CSVs in", results_dir)


Saved Nussinov evaluation CSVs in ..\results


## 6. Nussinov Baseline Run for ArchiveII

We implement:

- Dataset split into training, test and validation for ArchiveII dataset
- Run the baseline for the split sets


In [26]:
archiveii_csv_path = archiveii_csv_path  # from earlier


In [32]:
# -----------------------------
# Create ArchiveII 70/30 Train–Test Split
# -----------------------------
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# Load full ArchiveII dataset
full_df = pd.read_csv(archiveii_csv_path)
print("Full ArchiveII size:", len(full_df))

# 70% train, 30% test
train_df, test_df = train_test_split(
    full_df, test_size=0.30, random_state=42, shuffle=True
)

print("Train size:", len(train_df))
print("Test size:", len(test_df))

# Where to save (same directory as archiveII_from_bpseq.csv)
processed_dir = Path(archiveii_csv_path).parent

archive_train_csv = processed_dir / "archiveII_train.csv"
archive_test_csv  = processed_dir / "archiveII_test.csv"

train_df.to_csv(archive_train_csv, index=False)
test_df.to_csv(archive_test_csv, index=False)

print("Saved:")
print("  ", archive_train_csv)
print("  ", archive_test_csv)


Full ArchiveII size: 3975
Train size: 2782
Test size: 1193
Saved:
   c:\Fall_2025\CSE6301\Project\rna-structural-prediction\data\archiveII\processed\archiveII_train.csv
   c:\Fall_2025\CSE6301\Project\rna-structural-prediction\data\archiveII\processed\archiveII_test.csv


In [31]:
from torch.utils.data import DataLoader

# Datasets for each split
arch_train_dataset = ArchiveIIDataset(str(archive_train_csv))
arch_val_dataset   = ArchiveIIDataset(str(archive_val_csv))
arch_test_dataset  = ArchiveIIDataset(str(archive_test_csv))

print("Train dataset size:", len(arch_train_dataset))
print("Val dataset size:", len(arch_val_dataset))
print("Test dataset size:", len(arch_test_dataset))

# DataLoaders (models will see the FULL splits, not a subset)
train_loader = DataLoader(
    arch_train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_rna_batch,
)

val_loader = DataLoader(
    arch_val_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_rna_batch,
)

test_loader = DataLoader(
    arch_test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_rna_batch,
)

batch = next(iter(train_loader))
input_ids, mask, targets = batch
print("Train batch shapes:", input_ids.shape, mask.shape, targets.shape)


Train dataset size: 2782
Val dataset size: 357
Test dataset size: 836
Train batch shapes: torch.Size([8, 364]) torch.Size([8, 364]) torch.Size([8, 364, 364])
